# Environment Setup
## Import the libraries

In [1]:
import os
import tensorflow as tf

#data visualization packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

#keras packages
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

#model evaluation packages
from sklearn.metrics import f1_score, roc_auc_score, log_loss
from sklearn.model_selection import cross_val_score, cross_validate

dimension = 28

## Load dataset

In [ ]:
imgs_path = "Dataset/Train"
data = []
labels = []
classes = 43
for i in range(classes):
    img_path = os.path.join(imgs_path, str(i)) #0-42
    for img in os.listdir(img_path):
        im = Image.open(os.path.abspath(img_path + "/" + img))
        im = im.resize((dimension,dimension))
        im = im.convert('L')
        im = np.array(im)
        data.append(im)
        labels.append(i)
data = np.array(data)
labels = np.array(labels)
print("success")

# Data Preprocessing

## Data preparation

### Test-train split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print("Training shape: ",x_train.shape, y_train.shape)
print("Testing shape: ",x_test.shape, y_test.shape)

### Reshaping data

In [ ]:
x_train = x_train.reshape(31367, dimension*dimension)
x_test = x_test.reshape(7842, dimension*dimension)
print("Training shape: ",x_train.shape, y_train.shape)
print("Testing shape: ",x_test.shape, y_test.shape)

### Feature scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

#fit and transform training dataset
x_train = minmax.fit_transform(x_train)
#transform testing dataset
x_test = minmax.transform(x_test)

## Data Visualisation

In [ ]:
from random import randint
index = randint(0, len(x_train) - 1) # pick a random element from the train set
print(f"Pixel values range from {x_train[index].min()} to {x_train[index].max()}")
print(f"Shape of the image: {x_train[index].shape}")

In [ ]:
print('Number of unique classes: ', len(np.unique(y_train)))
print('Classes: ', np.unique(y_train))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5,figsize=(15, 5))
ax = axes.ravel()
for i in range(10):
    ax[i].imshow(x_train[i].reshape(dimension,dimension))
    ax[i].title.set_text('Class: ' + str(y_train[i]))
plt.subplots_adjust(hspace=0.5)
plt.show()

# Building Model, Validation and Testing

## Initialise Model

In [ ]:
#initializing model
model = Sequential()
#add 1st hidden layer
model.add(Dense(input_dim = x_train.shape[1], units = 256, kernel_initializer='uniform', activation='relu'))
#add output layer
model.add(Dense(units = len(np.unique(y_train)), kernel_initializer='uniform', activation='sigmoid'))
#compile the neural network
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model summary
model.summary()

## Train and validate

In [ ]:
history = model.fit(x_train, y_train, epochs=25, verbose=1, validation_data=(x_test, y_test))

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label="Training accuracy")
plt.plot(history.history['val_accuracy'], label="val accuracy")
plt.title("Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.figure(1)
plt.plot(history.history['loss'], label="training loss")
plt.plot(history.history['val_loss'], label="val loss")
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Testing

In [ ]:
from sklearn.metrics import accuracy_score

# Import images and labels from the test set
test = pd.read_csv("./Dataset/Test.csv")
test_labels = test['ClassId'].values
test_imgs = test['Path'].values
test_img_path = "./Dataset"
test_data = []
for img in test_imgs:
    im = Image.open(test_img_path + '/' + img)
    im = im.resize((dimension,dimension))
    im = im.convert('L')
    im = np.array(im)
    test_data.append(im)

# convert to numpy array
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# reshape the data
print("Testing shape: ",test_data.shape, test_labels.shape)
test_data = test_data.reshape(12630, dimension*dimension)
print("Testing shape: ",test_data.shape, test_labels.shape)

# normalize the data
test_data = minmax.transform(test_data)

# predict the results
predict_x=model.predict(test_data)

# select the index with the maximum probability
classes_x=np.argmax(predict_x,axis=1)

# print the accuracy
print("accuracy: ", accuracy_score(test_labels, classes_x))

# Extract Model weights

In [ ]:
np.savetxt('w1.txt', model.get_weights()[0] ,fmt='%1.4e', delimiter=',')

total = ""
with open("w1.txt", "r") as f:
    texts = f.readlines()
    for text in texts:
        total += str(text.split(",")).replace("[", "{").replace("]", "}")
        total += ", "
with open("w1-proc.txt", "w") as f:
    f.write(total)

model.get_weights()[2].shape

np.savetxt('w2.txt', model.get_weights()[2] ,fmt='%1.4e', delimiter=',')

total = ""
with open("w2.txt", "r") as f:
    texts = f.readlines()
    for text in texts:
        total += str(text.split(",")).replace("[", "{").replace("]", "}")
        total += ", "
with open("w2-proc.txt", "w") as f:
    f.write(total)
